In [1]:
import torch.nn as nn
import torch
import pytorch_lightning as pl
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score

In [2]:
df = pd.read_csv("loan_data.csv")
for x in df.columns:
    if "." in (x):
        df.rename(columns={x:x.replace(".","_")} , inplace = True)
df

,credit_policy,purpose,int_rate,installment,log_annual_inc,dti,fico,days_with_cr_line,revol_bal,revol_util,inq_last_6mths,delinq_2yrs,pub_rec,not_fully_paid
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9573,0,all_other,0.1461,344.76,12.180755,10.39,672,10474.000000,215372,82.1,2,0,0,1
9574,0,all_other,0.1253,257.70,11.141862,0.21,722,4380.000000,184,1.1,5,0,0,1
9575,0,debt_consolidation,0.1071,97.81,10.596635,13.09,687,3450.041667,10036,82.9,8,0,0,1
9576,0,home_improvement,0.1600,351.58,10.819778,19.18,692,1800.000000,0,3.2,5,0,0,1


In [3]:
categorical_columns = ['purpose']
numerical_columns = ['int_rate', 'installment', 'log_annual_inc',
       'dti', 'fico', 'days_with_cr_line', 'revol_bal', 'revol_util',
       'inq_last_6mths', 'delinq_2yrs', 'pub_rec']
label = "not_fully_paid"

In [4]:
df1 = df[df["not_fully_paid"] == 1].sample(1533)
df2 = df[df["not_fully_paid"] == 0].sample(1533)
dff = pd.concat([df1,df2] ,ignore_index = True)
df1 = df[df["not_fully_paid"] == 0].sample(1533)
df2 = df[df["not_fully_paid"] == 1].sample(1533)
dff = pd.concat([dff,df1,df2] ,ignore_index = True)
df1 = df[df["not_fully_paid"] == 0].sample(1533)
df2 = df[df["not_fully_paid"] == 1].sample(1533)
dff = pd.concat([dff,df1,df2] ,ignore_index = True)
df1 = df[df["not_fully_paid"] == 0].sample(1533)
df2 = df[df["not_fully_paid"] == 1].sample(1533)
dff = pd.concat([dff,df1,df2] ,ignore_index = True)
dff

,credit_policy,purpose,int_rate,installment,log_annual_inc,dti,fico,days_with_cr_line,revol_bal,revol_util,inq_last_6mths,delinq_2yrs,pub_rec,not_fully_paid
0,1,all_other,0.0894,47.66,10.596635,9.60,712,2610.041667,193,7.4,0,0,0,1
1,1,small_business,0.1635,482.25,10.404263,7.89,702,3510.041667,11209,66.7,1,0,0,1
2,1,all_other,0.1183,99.41,11.294769,23.67,697,7380.041667,17829,66.3,3,0,0,1
3,1,small_business,0.1916,183.69,10.858999,22.59,662,3300.000000,22312,81.1,0,0,0,1
4,1,debt_consolidation,0.1218,93.24,11.461632,16.96,697,8130.000000,40868,86.6,3,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12259,1,all_other,0.1221,116.61,10.858922,15.07,707,5580.000000,12324,25.5,2,0,0,1
12260,0,credit_card,0.1438,274.90,10.985293,23.86,677,7258.958333,14301,60.2,4,0,0,1
12261,1,all_other,0.1025,161.93,10.597634,19.12,717,1649.041667,10625,65.6,0,0,0,1
12262,0,home_improvement,0.1166,809.78,11.901583,18.43,722,13499.000000,247970,93.6,12,0,0,1


In [5]:
X = df[numerical_columns]
y = df[[label]]

In [6]:
train_x,test_x,train_y,test_y = train_test_split(X,y,test_size = 0.5,random_state = 42, stratify = y[label])
test_x,val_x,test_y,val_y = train_test_split(test_x,test_y,test_size = 0.5,random_state = 42 , stratify = test_y[label])

In [7]:
val_x

,int_rate,installment,log_annual_inc,dti,fico,days_with_cr_line,revol_bal,revol_util,inq_last_6mths,delinq_2yrs,pub_rec
3575,0.1158,165.07,10.545341,13.52,807,4589.958333,323,1.8,0,0,0
450,0.0743,217.52,10.404263,10.04,772,3630.000000,983,2.1,1,0,0
3186,0.1158,82.54,10.950807,19.87,697,4590.000000,17125,39.1,3,0,0
3663,0.0800,200.56,11.066576,0.00,807,3960.000000,0,0.0,1,0,0
1509,0.1399,854.32,11.695247,19.76,692,3780.041667,41077,60.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
8826,0.1588,140.40,10.341742,21.45,652,2613.000000,11157,54.7,2,0,0
8302,0.1387,119.41,10.126631,3.94,652,2202.000000,2728,48.7,3,0,0
6455,0.1287,50.45,10.968198,17.32,687,6032.041667,15333,94.6,1,0,0
2979,0.1316,515.00,11.184421,17.68,722,5310.000000,6759,0.0,3,0,0


In [8]:
class LoanDataset(Dataset):
    def __init__(self,features,labels):
        self.features = features
        self.labels = labels
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self,index):
        feature = self.features[index]
        label = self.labels[index]
        return torch.tensor(feature).float() , torch.tensor(label).float()

In [9]:
loans = LoanDataset(train_x.values , train_y.values)
features , labels = next(iter(DataLoader(loans,batch_size = 1 , shuffle = True)))
print(features,labels)

tensor([[1.2210e-01, 1.5992e+02, 1.1225e+01, 9.4100e+00, 6.9700e+02, 3.7500e+03,
         2.1069e+04, 8.7800e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00]]) tensor([[0.]])


In [10]:
class LoanDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.train = LoanDataset(train_x.values , train_y.values)
        self.test = LoanDataset(test_x.values , test_y.values)
        self.val = LoanDataset(val_x.values , val_y.values)
        
    def train_dataloader(self):
        return DataLoader(self.train,batch_size = 64 , shuffle = True)
    def test_dataloader(self):
        return DataLoader(self.test,batch_size = 32 , shuffle = False)
    def val_dataloader(self):
        return DataLoader(self.val,batch_size = 32 , shuffle = False)

In [11]:
loss_func  = torch.nn.BCEWithLogitsLoss()
sigmoid = nn.Sigmoid()
def metrics(y_pred,y):
    loss = loss_func(y_pred,y)
    y_pred = sigmoid(y_pred).detach()
    y_predictions = [1 if y_probs > 0.5 else 0 for y_probs in y_pred ]
    accuracy = accuracy_score(y,y_predictions) 
    precision = precision_score(y,y_predictions,zero_division=0,average='weighted')
    auc = roc_auc_score(y,y_pred)
    return accuracy,precision,auc,loss

In [16]:
class Loan_Model(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.initial_layer = nn.Sequential(
        nn.Linear(11,10),
        nn.Linear(10,11)
        )
        self.hidden1 = nn.Sequential(
        nn.Linear(11,50),
        )

        self.hidden2 = nn.Sequential(
            nn.Linear(50,25)
        )
        self.hidden3 = nn.Sequential(
            nn.Linear(25,75)
        )
        self.hidden4 = nn.Sequential(
            nn.Linear(75,50)
        )
        self.hidden5 = nn.Sequential(
            nn.Linear(50,25)
        )
        self.hidden6 = nn.Sequential(
            nn.Linear(25,50)
        )
        self.hidden7 = nn.Sequential(
            nn.Linear(50,25)
        )
        # self.hidden5 = nn.Sequential(
        #     nn.Linear(50,25)
        # )
        # self.hidden5 = nn.Sequential(
        #     nn.Linear(50,25)
        # )
        self.final_layer = nn.Linear(25,1)

    
    def forward(self,x):
        out = self.initial_layer(x)
        out = nn.functional.relu(out)
        out = self.hidden1(out)
        out = nn.functional.relu(out)
        out = self.hidden2(out)
        ut = nn.functional.relu(out)
        out = self.hidden3(out)
        out = nn.functional.relu(out)
        out = self.hidden4(out)
        out = nn.functional.relu(out)
        out = self.hidden5(out)
        out = nn.functional.relu(out)
        out = self.hidden6(out)
        out = nn.functional.relu(out)
        out = self.hidden7(out)
        out = self.final_layer(out)
        return out
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters() , lr = 1e-4)
        sch = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
        return {
            "optimizer" : optimizer,
            "lr_scheduler" :{
                "scheduler" : sch,
                "monitor" :"train_loss"
            }
        }
    
    def validation_step(self,batch,batch_index):
        x,y = batch
        y_pred = self(x)
        accuracy,precision,auc,loss = metrics(y_pred,y)
        self.log('auc', auc)
        return loss
    def test_step(self,batch,batch_index):
        x,y = batch
        y_pred = self(x)
        accuracy,precision,auc,loss = metrics(y_pred,y)
        self.log('accuracy', accuracy)
        self.log('precision', precision)
        self.log('auc', auc)
        return loss
    def training_step(self,batch,batch_index):
        x,y = batch
        y_pred = self(x)
        loss = loss_func(y_pred,y)
        self.log("train_loss" , loss)
        return loss

In [17]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
checkpoint_callback = ModelCheckpoint(monitor = "auc", mode = "max")
model = Loan_Model()
module = LoanDataModule()
trainer = Trainer(max_epochs = 30)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [18]:
trainer.fit(model,module)


  | Name          | Type       | Params
---------------------------------------------
0 | initial_layer | Sequential | 241   
1 | hidden1       | Sequential | 600   
2 | hidden2       | Sequential | 1.3 K 
3 | hidden3       | Sequential | 1.9 K 
4 | hidden4       | Sequential | 3.8 K 
5 | hidden5       | Sequential | 1.3 K 
6 | hidden6       | Sequential | 1.3 K 
7 | hidden7       | Sequential | 1.3 K 
8 | final_layer   | Linear     | 26    
---------------------------------------------
11.7 K    Trainable params
0         Non-trainable params
11.7 K    Total params
0.047     Total estimated model params size (MB)


Epoch 29: 100%|██████████| 150/150 [00:05<00:00, 25.76it/s, loss=0.451, v_num=15]


1

In [19]:
trainer.test()

Testing: 100%|██████████| 75/75 [00:01<00:00, 40.86it/s]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'accuracy': 0.8375104665756226,
 'auc': 0.5835332870483398,
 'precision': 0.8546363711357117}
--------------------------------------------------------------------------------


[{'accuracy': 0.8375104665756226,
  'precision': 0.8546363711357117,
  'auc': 0.5835332870483398}]